# Traitement de la base sirene

In [1]:
import os

import pandas as pd

In [2]:
# Ne pas oublier PEP 8

In [3]:
# Déclaration des constantes
DATA_DIR = '../data/external/'
PROCESSED_DIR = '../data/processed/'

ENTITIES_FILENAME = 'StockUniteLegale_utf8.zip'
ETAB_FILENAME = 'StockEtablissement_utf8.zip'
NAF_REF_FILENAME = 'naf86.xlsx'
PS_CITIES_FILENAME = 'PS_communes.csv'

ACTIVITY_CODE = '86.'

## Préparation des entités

In [4]:
pd.read_csv(os.path.join(DATA_DIR, ENTITIES_FILENAME), # Concaténation du répertoire et du nom de fichier
            compression='zip',
            #nrows=1000 # limite à 1000 lignes la lecture
           ).head()

C:\Users\p11749553\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3044: DtypeWarning: Columns (2,4,5,6,7,8,9,10,11,12,13,17,21,22,24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,siren,statutdiffusionunitelegale,unitepurgeeunitelegale,datecreationunitelegale,sigleunitelegale,sexeunitelegale,prenom1unitelegale,prenom2unitelegale,prenom3unitelegale,prenom4unitelegale,...,denominationunitelegale,denominationusuelle1unitelegale,denominationusuelle2unitelegale,denominationusuelle3unitelegale,categoriejuridiqueunitelegale,activiteprincipaleunitelegale,nomenclatureactiviteprincipaleunitelegale,nicsiegeunitelegale,economiesocialesolidaireunitelegale,caractereemployeurunitelegale
0,325175,O,NaN,2000-09-26,NaN,M,THIERRY,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1000,32.12Z,NAFRev2,65,NaN,N
1,1807254,O,NaN,1972-05-01,NaN,M,JACQUES-LUCIEN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1000,85.59A,NAFRev2,22,NaN,N
2,5410220,O,True,1954-12-25,NaN,M,GEORGES,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1000,22.02,NAP,15,NaN,N
3,5410345,O,True,NaN,NaN,M,MICHEL,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1000,79.06,NAP,10,NaN,N
4,5410394,O,True,1954-12-25,NaN,M,ROBERT,ALFRED,NaN,NaN,...,NaN,NaN,NaN,NaN,1000,64.42,NAP,18,NaN,N


In [5]:
pd.read_csv(os.path.join(DATA_DIR, ENTITIES_FILENAME), # Concaténation du répertoire et du nom de fichier
            compression='zip',
            nrows=1000 # limite à 1000 lignes la lecture
           ).columns

Index(['siren', 'statutdiffusionunitelegale', 'unitepurgeeunitelegale',
       'datecreationunitelegale', 'sigleunitelegale', 'sexeunitelegale',
       'prenom1unitelegale', 'prenom2unitelegale', 'prenom3unitelegale',
       'prenom4unitelegale', 'prenomusuelunitelegale', 'pseudonymeunitelegale',
       'identifiantassociationunitelegale', 'trancheeffectifsunitelegale',
       'anneeeffectifsunitelegale', 'datederniertraitementunitelegale',
       'nombreperiodesunitelegale', 'categorieentreprise',
       'anneecategorieentreprise', 'datedebut', 'etatadministratifunitelegale',
       'nomunitelegale', 'nomusageunitelegale', 'denominationunitelegale',
       'denominationusuelle1unitelegale', 'denominationusuelle2unitelegale',
       'denominationusuelle3unitelegale', 'categoriejuridiqueunitelegale',
       'activiteprincipaleunitelegale',
       'nomenclatureactiviteprincipaleunitelegale', 'nicsiegeunitelegale',
       'economiesocialesolidaireunitelegale', 'caractereemployeurunitelega

In [60]:
%%time
entities_df = (pd.read_csv(os.path.join(DATA_DIR, ENTITIES_FILENAME), # Concaténation du répertoire et du nom de fichier
                           usecols=['siren',
                                    'activiteprincipaleunitelegale', 'datecreationunitelegale',
                                    'denominationunitelegale', 'nomunitelegale', 'etatadministratifunitelegale',
                                   ],
                           compression='zip',
                           dtype={'codecommuneetablissement':str, 'codepostaletablissement':str}
                           #nrows=100000 # limite à n lignes la lecture
                          )
             ).dropna(subset=['activiteprincipaleunitelegale']).query('etatadministratifunitelegale == "A"')

# Filtrage des professionnels de santé
#entities_df = entities_df[entities_df.activiteprincipaleunitelegale.str.startswith(ACTIVITY_CODE)] # 86.

<string>:2: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.


Wall time: 51.4 s


In [7]:
entities_df.shape

(11696143, 6)

In [8]:
entities_df.denominationunitelegale.isnull().sum()

4403344

In [9]:
entities_df.nomunitelegale.isnull().sum()

7292817

In [10]:
entities_df.denominationunitelegale.isnull().sum() + entities_df.nomunitelegale.isnull().sum()

11696161

In [11]:
entities_df.head()

,siren,datecreationunitelegale,etatadministratifunitelegale,nomunitelegale,denominationunitelegale,activiteprincipaleunitelegale
0,325175,2000-09-26,A,JANOYER,NaN,32.12Z
31,5411582,1954-01-01,A,GERVOISE,NaN,68.20B
38,5420021,1954-01-01,A,NaN,ETABLISSEMENTS LUCIEN BIQUEZ,46.69B
45,5420120,1954-01-01,A,NaN,SOCIETE DES SUCRERIES DU MARQUENTERRE,70.10Z
51,5420260,1954-12-25,A,NaN,SOCIETE ANONYME MILBLED,20.01


## Préparation des établissements

In [12]:
pd.read_csv(os.path.join(DATA_DIR, ETAB_FILENAME), # Concaténation du répertoire et du nom de fichier
            compression='zip',
            nrows=1000 # limite à 1000 lignes la lecture
           ).head()

,siren,nic,siret,statutdiffusionetablissement,datecreationetablissement,trancheeffectifsetablissement,anneeeffectifsetablissement,activiteprincipaleregistremetiersetablissement,datederniertraitementetablissement,etablissementsiege,...,libellepaysetranger2etablissement,datedebut,etatadministratifetablissement,enseigne1etablissement,enseigne2etablissement,enseigne3etablissement,denominationusuelleetablissement,activiteprincipaleetablissement,nomenclatureactiviteprincipaleetablissement,caractereemployeuretablissement
0,325175,16,32517500016,O,2000-09-26,NaN,NaN,3212ZZ,2015-03-18T00:58:59,False,...,NaN,2009-05-27,F,NaN,NaN,NaN,NaN,32.12Z,NAFRev2,N
1,325175,24,32517500024,O,2008-05-20,NaN,NaN,NaN,2011-12-12T09:40:04,False,...,NaN,2011-10-21,F,TAHITI PERLES CREATIONS,NaN,NaN,NaN,47.89Z,NAFRev2,N
2,325175,32,32517500032,O,2009-05-27,NaN,NaN,NaN,2014-07-08T00:10:21,False,...,NaN,2011-10-21,F,TAHITI PERLES CREATIONS,NaN,NaN,NaN,32.12Z,NAFRev2,N
3,325175,40,32517500040,O,2011-10-21,NaN,NaN,3212ZZ,2015-03-18T00:58:59,False,...,NaN,2014-01-07,F,TAHITI PERLES CREATION,NaN,NaN,NaN,32.12Z,NAFRev2,N
4,325175,57,32517500057,O,2014-01-07,NaN,NaN,NaN,2018-07-10T14:17:15,False,...,NaN,2018-02-07,F,TAHITI PERLES CREATION,NaN,NaN,NaN,32.12Z,NAFRev2,N


In [13]:
pd.read_csv(os.path.join(DATA_DIR, ETAB_FILENAME), # Concaténation du répertoire et du nom de fichier
            compression='zip',
            nrows=1000 # limite à 1000 lignes la lecture
           ).columns.sort_values()

Index(['activiteprincipaleetablissement',
       'activiteprincipaleregistremetiersetablissement',
       'anneeeffectifsetablissement', 'caractereemployeuretablissement',
       'codecedex2etablissement', 'codecedexetablissement',
       'codecommune2etablissement', 'codecommuneetablissement',
       'codepaysetranger2etablissement', 'codepaysetrangeretablissement',
       'codepostal2etablissement', 'codepostaletablissement',
       'complementadresse2etablissement', 'complementadresseetablissement',
       'datecreationetablissement', 'datedebut',
       'datederniertraitementetablissement',
       'denominationusuelleetablissement',
       'distributionspeciale2etablissement',
       'distributionspecialeetablissement', 'enseigne1etablissement',
       'enseigne2etablissement', 'enseigne3etablissement',
       'etablissementsiege', 'etatadministratifetablissement',
       'indicerepetition2etablissement', 'indicerepetitionetablissement',
       'libellecedex2etablissement', 'libell

In [61]:
%%time
etab_df = (pd.read_csv(os.path.join(DATA_DIR, ETAB_FILENAME), # Concaténation du répertoire et du nom de fichier
                       usecols=['siren',
                                'activiteprincipaleetablissement', 'datecreationetablissement',
                                'enseigne1etablissement', 'denominationusuelleetablissement',
                                'etatadministratifetablissement',
                                'codepostaletablissement', 'codecommuneetablissement', 'libellecommuneetablissement',
                               ],
                           compression='zip',
                           dtype={'codecommuneetablissement':str, 'codepostaletablissement':str},
                           #nrows=1_000_000 # limite à n lignes la lecture
                          )
             ).dropna(subset=['activiteprincipaleetablissement']).query('etatadministratifetablissement == "A"')

# Filtrage des professionnels de santé
#etab_df = etab_df[etab_df.activiteprincipaleetablissement.str.startswith(ACTIVITY_CODE)] # 86.

<string>:2: DtypeWarning: Columns (41,44) have mixed types. Specify dtype option on import or set low_memory=False.


Wall time: 1min 32s


In [15]:
etab_df.shape

(11233968, 9)

In [16]:
etab_df.head(10)

,siren,datecreationetablissement,codepostaletablissement,libellecommuneetablissement,codecommuneetablissement,etatadministratifetablissement,enseigne1etablissement,denominationusuelleetablissement,activiteprincipaleetablissement
5,325175,2018-02-07,13004.0,MARSEILLE 4,13204,A,NaN,NaN,32.12Z
48,5420021,2009-12-23,80100.0,ABBEVILLE,80001,A,NaN,NaN,46.69B
56,5420120,1989-01-27,80120.0,RUE,80688,A,NaN,NaN,10.81Z
57,5420120,1900-01-01,62180.0,RANG DU FLIERS,62688,A,NaN,NaN,52.10B
58,5420120,1900-01-01,62140.0,MARCONNELLE,62550,A,NaN,NaN,70.10Z
59,5420120,1900-01-01,62180.0,CONCHIL LE TEMPLE,62233,A,NaN,NaN,46.75Z
71,5440110,1995-10-01,4000.0,DIGNE-LES-BAINS,04070,A,NaN,NaN,68.20B
79,5440565,1954-01-01,4200.0,SISTERON,04209,A,NaN,NaN,68.20B
119,5450093,2001-02-20,4800.0,GREOUX LES BAINS,04094,A,PASSIONNEMENT GLAMOUR,PASSIONNEMENT GLAMOUR,47.78C
120,5450093,2011-04-12,4800.0,GREOUX LES BAINS,04094,A,PASSIONNEMENT GLAMOUR,PASSIONNEMENT GLAMOUR,47.71Z


In [17]:
etab_df.dtypes

siren                                 int64
datecreationetablissement            object
codepostaletablissement             float64
libellecommuneetablissement          object
codecommuneetablissement             object
etatadministratifetablissement       object
enseigne1etablissement               object
denominationusuelleetablissement     object
activiteprincipaleetablissement      object
dtype: object

In [18]:
entities_df.dtypes

siren                             int64
datecreationunitelegale          object
etatadministratifunitelegale     object
nomunitelegale                   object
denominationunitelegale          object
activiteprincipaleunitelegale    object
dtype: object

In [52]:
process_df = etab_df.merge(entities_df, on='siren', how='left')

In [20]:
process_df.shape

(11233968, 14)

In [21]:
process_df[process_df.activiteprincipaleunitelegale.isnull()].shape

(23, 14)

In [22]:
etab_df.memory_usage(deep=True).sum() / (1024 * 1024) # En mégaoctet

4456.694915771484

In [23]:
entities_df.memory_usage(deep=True).sum() / (1024 * 1024) # En mégaoctet

3464.3372735977173

In [25]:
%load_ext dpaste_magic

In [62]:
#%%dpaste
merged_df = (etab_df.merge(entities_df, on='siren', how='outer', indicator=True))

ps_df = (merged_df.loc[merged_df.activiteprincipaleunitelegale.str.startswith("86.") |
                      merged_df.activiteprincipaleetablissement.str.startswith("86.")]
         .assign(etab_ape_l1 = lambda df: df.activiteprincipaleetablissement.str.slice(0, 2))
         .assign(el_ape_l1 = lambda x: x.activiteprincipaleunitelegale.str.slice(0, 2))
         .assign(cross_l1 = lambda df: df.el_ape_l1 + '_' + df.etab_ape_l1)
        )

In [54]:
pd.crosstab(ps_df.etab_ape_l1, ps_df.el_ape_l1)

el_ape_l1,01,02,03,05,10,11,13,14,15,18,...,84,85,86,87,88,90,93,94,95,96
etab_ape_l1,,,,,,,,,,,,,,,,,,,,,
00,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
01,0,0,0,0,0,0,0,0,0,0,...,0,0,1336,0,0,0,0,0,0,0
02,0,0,0,0,0,0,0,0,0,0,...,0,0,242,0,0,0,0,0,0,0
03,0,0,0,0,0,0,0,0,0,0,...,0,0,5,0,0,0,0,0,0,0
08,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,8,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,...,0,0,3,0,0,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,...,0,0,5,0,0,0,0,0,0,0


In [28]:
ps_df.shape

(590494, 18)

### Analyse des croisements NAF entités établissements

In [55]:
ps_df.cross_l1.value_counts(normalize=True).head()

86_86    0.949202
86_68    0.017824
84_86    0.005977
86_87    0.003883
86_88    0.002489
Name: cross_l1, dtype: float64

#### Cas des entités et établissements en 86.

In [56]:
pd.crosstab(ps_df.query('cross_l1 == "86_86"').activiteprincipaleetablissement,
            ps_df.query('cross_l1 == "86_86"').activiteprincipaleunitelegale)

activiteprincipaleunitelegale,86.10Z,86.21Z,86.22A,86.22B,86.22C,86.23Z,86.90A,86.90B,86.90C,86.90D,86.90E,86.90F
activiteprincipaleetablissement,,,,,,,,,,,,
86.10Z,8736,12,8,2,1,0,0,0,0,6,1,0
86.21Z,312,88879,202,68,423,15,0,6,6,19,9,10
86.22A,11,207,7618,0,100,0,0,0,0,1,0,0
86.22B,4,97,0,6410,106,6,0,0,0,0,0,1
86.22C,17,634,216,186,47899,12,0,10,0,1,2,8
86.23Z,9,4,0,16,7,44336,0,0,0,1,1,0
86.90A,35,0,0,0,0,0,7866,0,1,1,0,0
86.90B,15,10,3,0,24,0,0,6450,1,3,0,0
86.90C,14,0,0,0,0,0,0,0,422,0,0,0


### Préparation de la table des codes NAF

In [41]:
# A partir de https://www.insee.fr/fr/information/2120875
naf_df = pd.read_clipboard()
naf_df.to_excel(os.path.join(DATA_DIR, NAF_REF_FILENAME), index=False)

In [57]:
naf_df.head()

,NAF,Libellé,Catégorie
0,86.10Z,Activités hospitalières,hopital
1,86.21Z,Activité des médecins généralistes,generaliste
2,86.22A,Activités de radiodiagnostic et de radiothérapie,radiologiste
3,86.22B,Activités chirurgicales,chirurgien
4,86.22C,Autres activités des médecins spécialistes,autre_specialiste


In [43]:
# Première version
ps_df.loc[ps_df.cross_l1 == "86_86", 'code_activite'] = ps_df.activiteprincipaleetablissement
ps_df.loc[ps_df.el_ape_l1 != "86", 'code_activite'] = ps_df.activiteprincipaleetablissement
ps_df.loc[~ps_df.activiteprincipaleetablissement.str.startswith(ACTIVITY_CODE, na=False),
          'code_activite'] = ps_df.activiteprincipaleunitelegale

In [33]:
# Deuxième version
ps_df['code_activite'] = ps_df.apply(lambda x: x.activiteprincipaleetablissement
                                     if x.etab_ape_l1 == "86"
                                     else x.activiteprincipaleunitelegale,
                                     axis='columns'
                                    )

In [ ]:
# Troisième version
ps_df['code_activite'] = ps_df.activiteprincipaleetablissement
ps_df['code_activite'] = ps_df.where(ps_df.etab_ape_l1 != "86",
                                     ps_df.activiteprincipaleunitelegale
                                    )

In [64]:
# Quatrième version
import numpy as np

ps_df['code_activite'] = np.where(ps_df.etab_ape_l1 == '86',
                                  ps_df.activiteprincipaleetablissement,
                                  ps_df.activiteprincipaleunitelegale
                                 )

In [44]:
ps_df.code_activite.value_counts()

86.90D    133517
86.90E    133251
86.21Z     99310
86.90F     76535
86.22C     51951
86.23Z     47727
86.10Z     14214
86.90A      9205
86.22A      8573
86.22B      7139
86.90B      7031
86.06        463
86.90C       443
86.08        299
86.02        219
86.10        175
86.07        170
86.01        118
86.12         51
86.09         32
86.05         31
86.04         27
86.03          7
86.11          6
Name: code_activite, dtype: int64

### Ajout du type de PS & sauvegarde

In [65]:
(ps_df
 .merge(naf_df, how='inner', left_on='code_activite', right_on='NAF')
 [['codecommuneetablissement', 'Catégorie']]
 .dropna(subset=['codecommuneetablissement'])
 .assign(count = 1)
 .groupby(['codecommuneetablissement', 'Catégorie'])
 .sum()
 .reset_index()
 .pivot(index='codecommuneetablissement', columns='Catégorie', values='count')
 .fillna(0)
).to_csv(os.path.join(PROCESSED_DIR, PS_CITIES_FILENAME), sep=';')

## Vérification

In [50]:
check_df = pd.read_csv(os.path.join(PROCESSED_DIR, PS_CITIES_FILENAME),
                      sep=';',
                      dtype={'codecommuneetablissement':str}
                      )

In [ ]:
assert check_df.sum() == ps_df.codecommuneetablissement.nunique(), \
        'Nombre de communes différent'